In [1]:
import s3fs
import dask
from dask import delayed
from dask.distributed import Client, LocalCluster, progress, fire_and_forget

In [2]:
cluster = LocalCluster(n_workers=1, threads_per_worker=1)
client = Client(cluster)
cluster

VBox(children=(HTML(value='<h2>LocalCluster</h2>'), HBox(children=(HTML(value='\n<div>\n  <style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n  </style>\n  <table style="text-align: right;">\n    <tr><th>Workers</th> <td>1</td></tr>\n    <tr><th>Cores</th> <td>1</td></tr>\n    <tr><th>Memory</th> <td>2.15 GB</td></tr>\n  </table>\n</div>\n', layout=Layout(min_width='150px')), Accordion(children=(HBox(children=(IntText(value=0, description='Workers', layout=Layout(width='150px')), Button(description='Scale', layout=Layout(width='150px'), style=ButtonStyle()))), HBox(children=(IntText(value=0, description='Minimum', layout=Layout(width='150px')), IntText(value=0, description='Maximum', layout=Layout(width='150px')), Button(description='Adapt', layout=Layout(width='150px'), style=ButtonStyle())))), layout=Layout(min_width='500px'), selected_index=None, _titles={'0': 'Manual Scaling', '1': 'Adaptive Scaling'}))), HTML(value='<p><b>Dashboard: </b><a href="http://127.0.0.1:57639/status" target="_blank">http://127.0.0.1:57639/status</a></p>\n')))

In [3]:
class S3FsWrapper(object):
    def __init__(self):
        self.fs = s3fs.S3FileSystem()
    def get_s3fs(self):
        return self.fs

In [4]:
fs = S3FsWrapper()

In [5]:
%%time 
def fun(fs, x):
    return x + 1
tasks = [delayed(fun)(fs, i) for i in range(100000)]
fire_and_forget(client.compute(tasks))

while len(client.processing()) > 1:
    pass

distributed.core - WARNING - Event loop was unresponsive in Scheduler for 1.14s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - WARNING - Event loop was unresponsive in Nanny for 1.16s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - WARNING - Event loop was unresponsive in Nanny for 7.32s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - WARNING - Event loop was unresponsive in Scheduler for 7.32s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.


CPU times: user 2min 16s, sys: 3.39 s, total: 2min 19s
Wall time: 2min 18s


distributed.nanny - WARNING - Worker exceeded 95% memory budget.  Restarting
tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:57641, threads: 1>>
Traceback (most recent call last):
  File "/Users/andreas/anaconda3/lib/python3.6/site-packages/psutil/_psosx.py", line 348, in catch_zombie
    yield
  File "/Users/andreas/anaconda3/lib/python3.6/site-packages/psutil/_psosx.py", line 387, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot(self.pid)
ProcessLookupError: [Errno 3] No such process

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/andreas/anaconda3/lib/python3.6/site-packages/tornado/ioloop.py", line 1026, in _run
    return self.callback()
  File "/Users/andreas/tmp/distributed/distributed/nanny.py", line 254, in memory_monitor
    memory = psutil.Process(self.process.pid).memory_info().rss
  File "/Users/andreas/anaconda3/lib/python3.6